In [1]:
class LazyRangeList:
    '''an easy implementation of a list of ranges with the cost of relatively expensive iteration

    - easy implementation: internally elements are stored in a set, which simplifies almost all operations
    - expensive iteration: "list of ranges" is a view and comes to reality only when being iterated (lazy)

    say there are M ranges in which there are N elements/integers in total
        - spacial complexity: O(N)
        - insert/remove an element: O(1)
        - look up an element: O(1)
        - insert/remove a range: O(len(range))
        - find range for element: O(len(range))
    '''

    def __init__(self,
                 range2str = lambda r: '[%05d,%05d]' % (r.start, r.stop - 1),
                 list2str = lambda l: '〔 ' + ', '.join(l) + ' 〕'):
        self.elements = set()  # the model

        self.range_to_str = range2str
        self.list_to_str = list2str

    def _isolate_sequential_head(self, sorted_elements):
        for i, e in enumerate(sorted_elements):
            if i > 0 and sorted_elements[i - 1] + 1 != sorted_elements[i]:
                return (sorted_elements[:i], sorted_elements[i:])
        return (sorted_elements, [])

    def _make_list_of_lists(self, sorted_elements):
        head, tail = self._isolate_sequential_head(sorted_elements)
        return [] if len(head) == 0 else [head] + self._make_list_of_lists(tail)

    def _make_list_of_ranges(self, sorted_elements):
        return [range(l[0], l[-1] + 1) for l in self._make_list_of_lists(sorted_elements)]

    def __iter__(self):
        '''no optimization for now'''
        for r in self._make_list_of_ranges(sorted(self.elements)):
            yield r  # the view

    def __repr__(self):
        return self.list_to_str(self.range_to_str(r) for r in self)

    def __str__(self):
        return self.__repr__()

    def add_inclusive_range(self, lo, hi):
        self.add_range(range(lo, hi + 1))

    def remove_inclusive_range(self, lo, hi):
        self.remove_range(range(lo, hi + 1))

    def add_range(self, r):
        for element in r:
            self.elements.add(element)

    def remove_range(self, r):
        for i in r:
            self.elements.remove(i)

    def insert(self, i):
        self.elements.add(i)

    def remove(self, i):
        self.elements.remove(i)

    def contains(self, i):
        return i in self.elements

    def find(self, i):
        '''returns the range in which the element lives'''
        lo = self.find_low(i)
        hi = self.find_high(i)
        return range(lo, hi + 1)

    def find_low(self, i):
        return self.find_low_high(i, lambda k: k - 1)

    def find_high(self, i):
        return self.find_low_high(i, lambda k: k + 1)

    def find_low_high(self, i, op):
        if i not in self.elements:
            return None

        while True:
            j = op(i)
            if j not in self.elements:
                return i
            i = j

In [2]:
zipcodes = LazyRangeList()
zipcodes

〔  〕

In [3]:
zipcodes.add_inclusive_range(95000, 95033)
zipcodes

〔 [95000,95033] 〕

In [4]:
zipcodes.add_inclusive_range(98000, 98033)
zipcodes

〔 [95000,95033], [98000,98033] 〕

In [5]:
zipcodes.insert(8123)
zipcodes

〔 [08123,08123], [95000,95033], [98000,98033] 〕

In [6]:
zipcodes.insert(98003)
zipcodes

〔 [08123,08123], [95000,95033], [98000,98033] 〕

In [7]:
zipcodes.insert(98034)
zipcodes

〔 [08123,08123], [95000,95033], [98000,98034] 〕

In [8]:
zipcodes.insert(98122)
zipcodes

〔 [08123,08123], [95000,95033], [98000,98034], [98122,98122] 〕

In [9]:
zipcodes.remove(95003)
zipcodes

〔 [08123,08123], [95000,95002], [95004,95033], [98000,98034], [98122,98122] 〕

In [10]:
zipcodes.find(95001)

range(95000, 95003)

In [11]:
zipcodes.find(98034)

range(98000, 98035)

In [12]:
for r in zipcodes:
    print(r)

range(8123, 8124)
range(95000, 95003)
range(95004, 95034)
range(98000, 98035)
range(98122, 98123)
